<a href="https://colab.research.google.com/github/rithikkulkarni/ALDA-Course-Project/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initial preprocessing code

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/rithikkulkarni/ALDA-Course-Project/refs/heads/main/datasets/stock_tweets.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [3]:
!pip install transformers

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [5]:
from transformers import pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert", device="cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda


In [6]:
### Function for using FinBERT to label our data for supervised learning
def label_data(tweet_text):
  temp = pipe(tweet_text)
  if 'positive' in str(temp[0]):
    return 'positive'
  elif 'negative' in str(temp[0]):
    return 'negative'
  else:
    return 'neutral'

### Function for using FinBERT that should be faster because it processes multiple tweets in parallel
def label_data_batch(tweet_texts, batch_size=32):
    results = pipe(tweet_texts, batch_size=batch_size)
    labels = []
    for res in results:
        if 'positive' in str(res):
            labels.append('positive')
        elif 'negative' in str(res):
            labels.append('negative')
        else:
            labels.append('neutral')
    return labels

In [7]:
import torch
from datasets import Dataset

# Ensure GPU is enabled
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("text-classification", model="ProsusAI/finbert", device=device)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['Tweet']])  # Keep only relevant column

# Define a function to process tweets
def label_data_batch(batch):
    results = pipe(batch['Tweet'], batch_size=32)  # Adjust batch size
    return {'label': [res['label'].lower() for res in results]}  # Convert to lowercase

# Apply the function to the dataset
labeled_dataset = dataset.map(label_data_batch, batched=True)

# Convert back to DataFrame
df['label'] = labeled_dataset['label']

Device set to use cuda:0


Map:   0%|          | 0/80793 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
 4   label         80793 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [9]:
df.rename(columns={'label': 'Label'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
 4   Label         80793 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [10]:
df.Label.value_counts()

,count
Label,
neutral,62638
negative,9609
positive,8546


In [11]:
# Exporting labeled data
df.to_csv('labeled_stock_tweets.csv', index=False)

In [23]:
# Importing labeled data
labeled_df = pd.read_csv('labeled_stock_tweets.csv')
labeled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
 4   Label         80793 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [24]:
# Basic tweet cleaning function, subject to change!

import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Define our stopwords from nltk
stop_words = set(stopwords.words('english'))

def clean_tweet(tweet, remove_stopwords=True):
    # Make everything lowercase
    tweet = tweet.lower()

    # Remove all URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)

    # Replace @ mentions with just the username
    tweet = re.sub(r'@\w+', 'USER', tweet)

    # Remove hashtags but keep the words from the hashtags
    tweet = re.sub(r'#', '', tweet)

    # Remove any numbers (helps generalization and makes sentiment analysis easier)
    tweet = re.sub(r'\d+', '', tweet)

    # Remove special characters and any punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Shrink any words that have overused repeated letters back down into their original word
    tweet = re.sub(r'(.)\1{2,}', r'\1', tweet)

    # Tokenize all the words
    words = tweet.split()

    # Remove any stopwords (words like the, a, is because they have negligible weight for sentiment analysis)
    if remove_stopwords:
        words = [word for word in words if word not in stop_words]

    # Put everything back together
    return ' '.join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
labeled_df['Cleaned_Tweet'] = labeled_df['Tweet'].apply(lambda x: clean_tweet(str(x)))

In [ ]:
labeled_df.head(50)

In [ ]:
# ### To be used for stock price

# import yfinance as yf

# ticker = yf.Ticker("AAPL")
# hist = ticker.history(start="2024-02-05", end="2024-02-12")
# print(hist)

                                 Open        High         Low       Close  \
Date                                                                        
2024-02-05 00:00:00-05:00  187.027102  188.120543  184.730891  186.559906   
2024-02-06 00:00:00-05:00  185.744802  188.180177  185.655343  188.170242   
2024-02-07 00:00:00-05:00  189.502257  189.909814  187.484373  188.279602   
2024-02-08 00:00:00-05:00  188.259697  188.408796  186.231879  187.196091   
2024-02-09 00:00:00-05:00  187.763391  189.097105  187.116452  187.962463   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-02-05 00:00:00-05:00  69668800       0.00           0.0  
2024-02-06 00:00:00-05:00  43490800       0.00           0.0  
2024-02-07 00:00:00-05:00  53439000       0.00           0.0  
2024-02-08 00:00:00-05:00  40962000       0.00           0.0  
2024-02-09 00:00:00-05:00  45155200       0.24           0.0  
